In [21]:
import pandas as pd
from scipy.spatial import distance
from sqlalchemy import create_engine

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [22]:
df_all_features = pd.DataFrame(df_feature_vector.columns.to_list(), columns=['name'])
df_merge_tag_features = pd.merge(df_tag, df_all_features, on='name')

allFeatures = df_feature_vector.columns.to_list()
df_name = df_merge_tag_features['name']

for i in df_name.to_numpy():
    allFeatures.remove(i)

for i in allFeatures:
    df_merge_tag_features = df_merge_tag_features.append({'name': i, 'weight': 0.1}, ignore_index=True)

array = df_feature_vector.columns.to_list()
array.remove('id')
array.remove('Mit Abstand')
array.remove('Videokonferenz')
array.remove('Pfadfinder')
array.remove('Rover')

weight = df_merge_tag_features['weight'].to_numpy()
array_labels = df_merge_tag_features['name'].to_numpy()

In [23]:
heimabend_index = 5

for heimabend_index in range(1, 200):
    db = create_engine('sqlite:////Users/rob/Documents/privat/Pfadfinder/Inspirator/source_code/heimabend/backend/heimabend/db.sqlite3')

    df_tag = pd.read_parquet('data/tag.parquet')
    df_tag = df_tag[['name', 'category_id']]
    df_tag['weight'] = df_tag['category_id'].apply(lambda x: 3 if x == 9 else 0.1 if x == 1 else 1)

    df_feature_vector = pd.read_parquet('data/feature_vector_num.parquet')
    df_feature_vector_with_text = pd.read_parquet('data/feature_vector.parquet')

    array_labels = df_feature_vector.columns.to_list()

    df_feature_vector_with_text['index'] = df_feature_vector_with_text.index
    title_array = df_feature_vector_with_text[['id', 'title']]
    title_list = title_array.values.tolist()

    feature_list = df_feature_vector[array_labels].values.tolist()

    ary_1 = feature_list[heimabend_index]

    feature_ary = []
    for index, element in enumerate(feature_list):
        score_cosine = distance.cosine(ary_1, element, weight)
        feature_ary.append([score_cosine])

    df_1 = pd.DataFrame(feature_ary, columns=['score'])
    df_2 = pd.DataFrame(title_list, columns=['event_score_id', 'title'])
    df = pd.concat([df_1, df_2], axis=1,).sort_values(by=['score']).iloc[1:15]
    df['event_id'] = df_feature_vector_with_text.iloc[heimabend_index]['id']
    df[['event_id', 'event_score_id', 'score']].to_sql('basic_nextbestheimabend', con=db, if_exists='append', index=False)
    print(df[['score', 'title', 'event_score_id', 'event_id']])

1       194
128  0.000024         Spiel: Der Kaiser mag...             218       194
127  0.000027              Naturbilder basteln             217       194
98   0.000027                Luftballonstaffel             162       194
129  0.000029                       Naturbingo             219       194
119  0.000032   Pfadfinder*innen ohne Plastik?             200       194
117  0.000034         Eigene Löffel schnitzen!             198       194
131  0.000036                  Wasserdetektive             221       194
102  0.000037                     Kerzendrehen             173       194
126  0.000038                Ich male ein Bild             216       194
97   0.000039             Kerzenwachs recyceln             161       194
130  0.000040  Upcycling - Reisespiele basteln             220       194
        score                              title  event_score_id  event_id
133  0.000026                    Pappteller-Golf             223       198
132  0.000027                    Di

IndexError: list index out of range